In [1]:
import os
import pandas as pd
import numpy as np 
import PyPDF2
import string
import nltk
from nltk.tokenize.regexp import regexp_tokenize
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import re
from nltk.util import ngrams
import math
import pickle

In [2]:
path = "r_s_test"
docs = os.listdir(path)
n = len(docs)

exclude = set(string.punctuation)
docslist = []
alldocslist = []
for index, i in  enumerate(docs):
    doctext = []
    if (i.endswith(".pdf")) or (i.endswith(".PDF")):
        in_file = os.path.abspath(path + "\\" + i)
        in_file1 = in_file.encode('UTF-8')
        pdfileobj = open(in_file1,'rb')
        pdfreader = PyPDF2.PdfFileReader(pdfileobj)   
        count = pdfreader.numPages
        for j in range(count):
            page = pdfreader.getPage(j)
            pp = page.extractText()
            doctext.append(pp)
            
        dt = ''.join(ch for ch in doctext)
        docslist.append(dt)
        doctext = ''.join(ch for ch in doctext if ch not in exclude)        
        alldocslist.append(doctext)  


In [3]:
plot_data1 = [[]] * len(alldocslist)
plot_data2 = [[]] * len(alldocslist)
plot_data3 = [[]] * len(alldocslist)

for doc in alldocslist:
    doctext = doc
    tokentext = regexp_tokenize(doctext, pattern = '\s+', gaps = True)
    plot_data1[index].append(tokentext)
    plot_data2[index].append(tokentext)
    plot_data3[index].append(tokentext)

In [4]:
def text_summarizer(docx, sentN):
    #docx = re.sub(r"[0-9]+", 'x', docx)
    parser = PlaintextParser.from_string(docx, Tokenizer(language = "english"))
    summarizer_lsa = LsaSummarizer()
    summary = summarizer_lsa(parser.document, sentN)
#    for sentences in summary:
#        print(sentences)
   # textsumm = ' '.join(str(sentences) for sentences in summary) + '.'
    textsumm = ' '.join(str(sentences) for sentences in summary)
    return textsumm

text_data = [[]] * len(docslist)
for doc in docslist:
    doctext = doc
    txtsum = text_summarizer(doctext, 2)
    text_data[index].append(txtsum)

In [5]:
for x in range(len(docs)):  
    unigrams = list(ngrams(plot_data1[0][x], 1))
    grams1 = [' '.join(unigrams[n]) for n in range(len(unigrams))]
    
    bigrams = list(ngrams(plot_data1[0][x], 2))
    grams2 = [' '.join(bigrams[n]) for n in range(len(bigrams))]
    
    trigrams = list(ngrams(plot_data1[0][x], 3))
    grams3 = [' '.join(trigrams[n]) for n in range(len(trigrams))]
    
    plot_data1[0][x] = grams1 
    plot_data2[0][x] = grams2
    plot_data3[0][x] = grams3 


l1 = plot_data1[0]
l2 = plot_data2[0]
l3 = plot_data3[0]

flatten1 = [item for sublist in l1 for item in sublist]
flatten2 = [item for sublist in l2 for item in sublist]
flatten3 = [item for sublist in l3 for item in sublist]
words1 = flatten1
words2 = flatten2
words3 = flatten3

wordsunique1 = set(words1) 
wordsunique1 = list(wordsunique1)
wordsunique2 = set(words2) 
wordsunique2 = list(wordsunique2)
wordsunique3 = set(words3) 
wordsunique3 = list(wordsunique3)

In [6]:
def tf(word, doc):
    return doc.count(word)/len(doc)

def containing(word, doclist):
    return sum(1 for doc in doclist if word in doc)

def idf(word, doclist):
    return math.log(len(doclist)/(containing(word, doclist)))

def tfidf(word, doc, doclist):
    counter = (tf(word, doc)*idf(word, doclist)) * 100
    return (round(counter, 2))

In [7]:
plottest = text_data[0][0:n]

#UNIGRAMS
plottest1 = plot_data1[0][0:n]
#t0 = round((time.clock())/60, 2)
search_word = {}
for doc in plottest1:
    for word in wordsunique1:
        if word in doc:
            word = str(word)
            index = plottest1.index(doc)
            doc_name = docs[index]
            idfs = tfidf(word, doc, plottest1)
            try:
                search_word[word].append([doc_name, plottest[index], idfs])
            except:
                search_word[word] = []
                search_word[word].append([doc_name, plottest[index], idfs])


#BIGRAMS
plottest2 = plot_data2[0][0:n]
#plottest = text_data[0][0:n]
#t0 = round((time.clock())/60, 2)
search_word1 = {}
for doc in plottest2:
    for word in wordsunique2:
        if word in doc:
            word = str(word)
            index = plottest2.index(doc)
            doc_name = docs[index]
            idfs = tfidf(word, doc, plottest2)
            try:
                search_word1[word].append([doc_name, plottest[index], idfs])
            except:
                search_word1[word] = []
                search_word1[word].append([doc_name, plottest[index], idfs])


search_word.update(search_word1)

#TRIGRAMS
plottest3 = plot_data3[0][0:n]
#plottest = text_data[0][0:n]
#t0 = round((time.clock())/60, 2)
search_word2 = {}
for doc in plottest3:
    for word in wordsunique3:
        if word in doc:
            word = str(word)
            index = plottest3.index(doc)
            doc_name = docs[index]
            idfs = tfidf(word, doc, plottest3)
            try:
                search_word2[word].append([doc_name, plottest[index], idfs])
            except:
                search_word2[word] = []
                search_word2[word].append([doc_name, plottest[index], idfs])


search_word.update(search_word2)

In [8]:
filename = "test.pkl"
fileobj = open(filename,'wb')
pickle.dump(search_word, fileobj)
fileobj.close()